https://blog.quantinsti.com/heston-model/

# Heston 1993 Model

## Vanilla Option 

In [45]:
import numpy as np

def heston_charfunc(u, params):
    S0   = params["S0"]
    K    = params["K"]
    r    = params["r"]
    q    = params["q"]
    v0   = params["v0"]
    kappa= params["kappa"]
    theta= params["theta"]
    sigma= params["sigma"]
    rho  = params["rho"]
    T    = params["T"]

    iu = 1j * u

    alpha = -0.5 * (u**2 + iu)
    beta  = kappa - rho * sigma * iu
    gamma = 0.5 * sigma**2

    d = np.sqrt(beta**2 - 4.0 * alpha * gamma)
    g = (beta - d) / (beta + d)

    exp_dT = np.exp(-d * T)

    C = (r - q) * iu * T + \
        (kappa * theta / gamma) * ((beta - d) * T - 2.0 * np.log((1.0 - g * exp_dT) / (1.0 - g)))

    D = (beta - d) / gamma * ((1.0 - exp_dT) / (1.0 - g * exp_dT))

    return np.exp(C + D * v0 + iu * np.log(S0 * np.exp(-q * T)))


def heston_price_call_fft(params, N=10000, U_max=1000):

    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)  # avoid u=0 singularity
    du = u[1] - u[0]
    lnK = np.log(K)

    phi_u     = heston_charfunc(u, params)
    phi_u_im1 = heston_charfunc(u - 1j, params)
    phi_im1   = heston_charfunc(-1j, params)

    integrand_P1 = np.real(np.exp(-1j * u * lnK) * phi_u_im1 /
                           (1j * u * phi_im1))
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u /
                           (1j * u))

    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1)
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put = K*np.exp(-r*T)*(1-P2) - S0*np.exp(-q*T)*(1-P1)
    return call, put

In [62]:
# Example
params = {
    "S0": 3795.74,
    "K" : 3795.74,
    "r":  0.014,
    "q": 0.015,
    "v0": 0.0112,
    "kappa": 0.2183,
    "theta": 0.0601,
    "sigma": 0.3171,
    "rho": -0.5250,
    "T": 1.0
}

call, put = heston_price_call_fft(params)
print(f"Call Price: {call:.4f}")
print(f"Put Price: {put:.4f}")

Call Price: 237.4693
Put Price: 241.2104


In [63]:
bs_call_price(3795.74,3795.74,0.014,0.015,0.1639,1.0)

np.float64(242.4783779478173)

## Extract Data before calibrating the model

In [30]:
import pandas as pd

# 1) On définit d’abord les maturités (index) et les niveaux de strikes (colonnes)
maturities = ['T', '1M', '2M', '3M', '6M', '9M', '1Y', '18M', '2Y', '3Y', '4Y', '5Y', '7Y', '10Y']
strikes_pct = ['80%', '90%', '95%', '97.5%', '100%', '102.5%', '105%', '110%', '120%']

# 2) On crée ensuite la liste des listes « raw » correspondant aux lignes du tableau
data = [
    # Strike prices (ligne 'T')
    [3036.59, 3416.17, 3605.95, 3700.85, 3795.74, 3890.63, 3985.53, 4175.31, 4554.89],
    # Valeurs pour 1M
    [25.29,   19.15,   15.35,   13.70,   12.25,   11.04,   10.71,   11.46,   13.56],
    # 2M
    [24.28,   18.46,   15.81,   14.58,   13.43,   12.39,   11.67,   11.52,   12.55],
    # 3M
    [23.03,   18.08,   15.96,   14.96,   14.00,   13.10,   12.38,   11.83,   12.06],
    # 6M
    [21.93,   18.24,   16.61,   15.85,   15.13,   14.47,   13.86,   12.87,   12.40],
    # 9M
    [21.22,   18.35,   17.05,   16.41,   15.79,   15.20,   14.65,   13.71,   12.72],
    # 1Y
    [21.14,   18.61,   17.48,   16.93,   16.39,   15.86,   15.35,   14.43,   13.28],
    # 18M
    [21.15,   19.15,   18.18,   17.71,   17.24,   16.79,   16.36,   15.58,   14.39],
    # 2Y
    [21.03,   19.33,   18.51,   18.10,   17.71,   17.33,   16.97,   16.30,   15.22],
    # 3Y
    [21.19,   19.68,   18.95,   18.59,   18.25,   17.91,   17.59,   16.99,   15.98],
    # 4Y
    [21.29,   19.99,   19.36,   19.06,   18.77,   18.49,   18.22,   17.72,   16.84],
    # 5Y
    [21.52,   20.36,   19.82,   19.56,   19.31,   19.06,   18.82,   18.38,   17.60],
    # 7Y
    [22.14,   21.18,   20.74,   20.53,   20.33,   20.13,   19.94,   19.58,   18.93],
    # 10Y
    [23.17,   22.41,   22.06,   21.89,   21.73,   21.57,   21.42,   21.14,   20.62],
]

# 3) On construit le DataFrame
df = pd.DataFrame(data, index=maturities, columns=strikes_pct)

# 4) Optionnel : convertir en float (ici déjà float)
df = df.astype(float)

print(df)


         80%      90%      95%    97.5%     100%   102.5%     105%     110%  \
T    3036.59  3416.17  3605.95  3700.85  3795.74  3890.63  3985.53  4175.31   
1M     25.29    19.15    15.35    13.70    12.25    11.04    10.71    11.46   
2M     24.28    18.46    15.81    14.58    13.43    12.39    11.67    11.52   
3M     23.03    18.08    15.96    14.96    14.00    13.10    12.38    11.83   
6M     21.93    18.24    16.61    15.85    15.13    14.47    13.86    12.87   
9M     21.22    18.35    17.05    16.41    15.79    15.20    14.65    13.71   
1Y     21.14    18.61    17.48    16.93    16.39    15.86    15.35    14.43   
18M    21.15    19.15    18.18    17.71    17.24    16.79    16.36    15.58   
2Y     21.03    19.33    18.51    18.10    17.71    17.33    16.97    16.30   
3Y     21.19    19.68    18.95    18.59    18.25    17.91    17.59    16.99   
4Y     21.29    19.99    19.36    19.06    18.77    18.49    18.22    17.72   
5Y     21.52    20.36    19.82    19.56    19.31    

In [36]:
df.columns

Index(['80%', '90%', '95%', '97.5%', '100%', '102.5%', '105%', '110%', '120%'], dtype='object')

## Calibration

In [66]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

# --- 1) Black–Scholes call price pour reconstruire les prix marchés à partir des vols ---

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


# --- 2) Extraction des données du DataFrame ------------------------------------------------

# On suppose que votre df ressemble à ceci (cf. message précédent) :
#   - index : maturities (str) avec une ligne 'T' qui contient les strikes
#   - colonnes : niveaux de strike en pourcentage ('80%', '90%', …, '120%')
#   - df.loc['T', k] = strike absolu ; df.loc[maturity, k] = vol implicite

# Constantes connues
S0 = 3795.74
r  = 0.014   # 1.40% continu
q  = 0.015   # 1.50% continu

# Conversion des labels de maturités en années
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(label)

# Construction d’une liste de points marché (T, K, prix_call)
market_data = []
strikes_abs = df.loc['T']  # Série indexée par '80%',…,'120%'
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K      = strikes_abs[pct]
        vol_imp= df.loc[mat, pct]/100.0  # en pourcentage
        price  = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data, 
                       dtype=[('T', float), ('K', float), ('price', float)])


# --- 3) Fonction objectif pour l’optimiseur -----------------------------------------------

def pack_params(x):
    """Dépaquète le vecteur x en dictionnaire de paramètres Heston."""
    return {
        "S0":   S0,
        "r":    r,
        "q":    q,
        "v0":   x[0],
        "kappa":x[1],
        "theta":x[2],
        "sigma":x[3],
        "rho":  x[4],
    }

def residuals(x):
    """Retourne les écarts modèle – marché pour chaque point de la surface."""
    params_base = pack_params(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = heston_price_call_fft(params, N=1000, U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)


# --- 4) Calibration par moindres carrés non‑linéaires -------------------------------------

# 4.1) Initial guess and bounds
x0 = np.array([
    0.04,    # v0
    1.0,     # kappa
    0.04,    # theta
    0.5,     # sigma
    -0.5,    # rho
])

# on impose rho ∈ [−0.999, 0.999] et les autres ≥ 0
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999]
ub = [2.0, 10.0, 2.0, 5.0, 0.999]

opt = least_squares(
    residuals,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=200
)

# 4.2) Résultats
v0_fit, kappa_fit, theta_fit, sigma_fit, rho_fit = opt.x
print("Paramètres calibrés Heston :")
print(f" v0    = {v0_fit:.4f}")
print(f" kappa = {kappa_fit:.4f}")
print(f" theta = {theta_fit:.4f}")
print(f" sigma = {sigma_fit:.4f}")
print(f" rho   = {rho_fit:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         7.2851e+05                                    7.14e+06    
       1              2         7.5827e+04      6.53e+05       4.84e-01       6.49e+05    
       2              3         1.6182e+04      5.96e+04       3.73e-01       1.58e+05    
       3              4         7.9612e+03      8.22e+03       2.52e-02       1.53e+05    
       4              5         4.0001e+03      3.96e+03       1.09e+00       9.55e+05    
       5              7         2.1285e+03      1.87e+03       7.44e-02       3.60e+05    
       6              8         1.9132e+03      2.15e+02       9.11e-02       1.62e+06    
       7              9         1.5982e+03      3.15e+02       4.19e-02       4.01e+05    
       8             10         1.5508e+03      4.74e+01       6.26e-02       1.04e+06    
       9             11         1.4553e+03      9.55e+01       1.52e-02       1.51e+05    

In [68]:
# Test de la calibration
# On compare les prix Heston et Black-Scholes
print("Prix Black-Sholes, M=1Y, S=100%:", bs_call_price(3795.74,3795.74,0.014,0.015,0.1639,1.0))
params = {
    "S0": 3795.74,
    "K" : 3795.74,
    "r":  0.014,
    "q": 0.015,
    "v0": 0.0112,
    "kappa": 0.2183,
    "theta": 0.0601,
    "sigma": 0.3171,
    "rho": -0.5250,
    "T": 1.0
}

call, _ = heston_price_call_fft(params)
print("Prix Heston, M=1Y, S=100%:", heston_price_call_fft(params, N=1000, U_max=2000)[0])

Prix Black-Sholes, M=1Y, S=100%: 242.4783779478173
Prix Heston, M=1Y, S=100%: 237.43623474640322


# Heston Bates 1996